# Overview

This notebook loads the base model `unsloth/gemma-3n-E4B-it` and merges it with adapters trained using Unsloth from the repository `ericrisco/medical-gemma-3n`.

The adapters were fine-tuned for 1 epoch using the `ericrisco/medrescue` dataset — a synthetic, instruction-based dataset focused on first aid, emergency response, and clinical reasoning.

The result is a fully merged Gemma-3n model ready for offline inference and deployment, with no need for separate LoRA adapters.


In [ ]:
!pip install -U transformers peft huggingface_hub accelerate

## Load and Merge Adapters
Load Unsloth-trained adapters from Hugging Face and merge them with the base Gemma-3n model.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "unsloth/gemma-3n-E4B-it"
ADAPTER_MODEL = "ericrisco/medical-gemma-3n-lora"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(ADAPTER_MODEL, use_fast=False)
peft_model = PeftModel.from_pretrained(base_model, ADAPTER_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
merged_model = peft_model.merge_and_unload()
print(type(merged_model))

<class 'transformers.models.gemma3n.modeling_gemma3n.Gemma3nForConditionalGeneration'>


In [ ]:
merged_model.eval()

prompt = "Pregunta: Me ha picado una serpiente\nRespuesta:"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(merged_model.device) for k, v in inputs.items()}

with torch.no_grad():
    output_ids = merged_model.generate(**inputs, max_new_tokens=50)

respuesta = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(respuesta)

Pregunta: Me ha picado una serpiente
Respuesta: Quédate quieto, aplica hielo o un paño frío, no intentes succionar el veneno ni cortar la piel, y busca atención médica de inmediato.

Aquí hay algunas consideraciones adicionales:

*   ¿Qué tipo de serpiente


## Load and Merge Adapters
Load Unsloth-trained adapters from Hugging Face and merge them with the base Gemma-3n model.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

repo_id = "ericrisco/medical-gemma-3n-4b"

In [ ]:
merged_model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ericrisco/medical-gemma-3n-4b/commit/e793db4ddbef49140772dfae9c6e4484e7a47e9a', commit_message='Upload tokenizer', commit_description='', oid='e793db4ddbef49140772dfae9c6e4484e7a47e9a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ericrisco/medical-gemma-3n-4b', endpoint='https://huggingface.co', repo_type='model', repo_id='ericrisco/medical-gemma-3n-4b'), pr_revision=None, pr_num=None)